<a href="https://colab.research.google.com/github/kethshen/FYP/blob/main/Energyplus_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ----- 1. Prep background -----

#### 1.1 intall utility from github repo

In [ ]:
# install from dev branch github
!pip install -q "energy-plus-utility @ git+https://github.com/mugalan/energy-plus-utility.git@dev"

#### 1.2 ready colab by installs deps, fetches E+, sets env/paths

In [ ]:
# run the silent bootstrap in this kernel
from eplus import prepare_colab_eplus
prepare_colab_eplus()  # raises on failure, otherwise silent

# import main tools
from eplus import EPlusUtil, EPlusSqlExplorer

#### 1.3 import ,idf .epw / setup output folder

In [ ]:
import subprocess, json, pathlib, os
import pandas as pd

# where EnergyPlus is installed
EPLUS = str(pathlib.Path.home() / "EnergyPlus-25-1-0")
EPLUS_ROOT = "/root/EnergyPlus-25-1-0"

# output folder and model files .idf and .epw
out_dir = "/content/eplus_out"
idf = f"{EPLUS}/ExampleFiles/5ZoneAirCooled.idf"
epw = f"{EPLUS}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"

#### 1.4 create EPlusUtil object

In [ ]:
util = EPlusUtil(verbose=1, out_dir = out_dir) #print outputs, where results will be saved
util.delete_out_dir() #clean if previous outputs exists

util.set_model(idf, # which building to analyse
               epw, # which weather to use
               outdoor_co2_ppm=400.0, #outdoor CO2 level
               per_person_m3ps_per_W=3.82e-08) #conversion factor for occupancy to CO2

util.ensure_output_sqlite() # Forces EnergyPlus to write `eplusout.sql`
util.enable_runtime_logging() # Turns on detailed logging during simulation

#### 1.4 Test callback method

In [ ]:
# A simple test function
# Mimics how EKF or controller callbacks work

def test_method (self,s,aa): #`self` → `EPlusUtil`/ `s` → EnergyPlus runtime state / `aa` → test argument
    print('inside test',aa)
    return aa

In [ ]:
# Dynamically adds `test_method` to the `util` object
# This is exactly how EKF and controller handlers are attached
# inject my own logic into EnergyPlus at runtime

import types
util.test_method = types.MethodType(test_method, util)

In [ ]:
# Call the test method
# to verify method binding works and dynamic callbacks are functional

util.test_method(0,1)

#### 1.5 convert .idf to .epJSON

In [ ]:
# get .idf file path
# Creates a path object pointing to the building model file.

idf_path = pathlib.Path(idf)

In [ ]:
# `ConvertInputFormat.exe` is an EnergyPlus tool
# It converts between .idf / .epJSON / older EnergyPlus formats

converter = os.path.join(EPLUS_ROOT, "ConvertInputFormat")

In [ ]:
# run the conversion
# Calls the EnergyPlus converter from Python and convert
# `check=True` → throws an error if conversion fails

subprocess.run([converter, str(idf_path)], check=True)

In [ ]:
# check converted sucess and .epJSON exists

epjson_path = idf_path.with_suffix(".epJSON")
print("epJSON exists?", epjson_path.exists(), epjson_path)

## ----- 2. Prep simulation -----

#### 2.1 Dry run

In [ ]:
util = EPlusUtil(verbose=1) # create object
util.delete_out_dir() # delete existing output results
util.set_model(idf,epw) # import building and weather files
util.ensure_output_sqlite() # enable generate eplusout.sql

# Runs EnergyPlus without full simulation - dry run
# doesn't simulate a year
# just create variable and actuator dictionaries , SQL tables
util.dry_run_min(include_ems_edd=False)

#### 2.2 List all the zones in the building

In [ ]:
util.list_zone_names()

#### 2.3 list all variable - can be measured (sensors)

In [ ]:
# zone temps, humid ratios, CO2 conc. , node temps, mass flow rates, etc...

util.list_available_variables()

#### 2.4 list all variable - can be controlled (actuators)

In [ ]:
# VAV damper position, Supply fan availability, Schedule values, Node mass flow setpoints

actuators_df=util.list_available_actuators()
actuators_df

In [ ]:
# Finds actuators related to VAV systems
# Assuming you have a DataFrame named 'df' with a column named 'ColumnName'
# Filter rows where 'ColumnName' contains 'VAV'

filtered_df = actuators_df[actuators_df['ActuatorKey'].str.contains('VAV', na=False)]

# Display the filtered DataFrame
display(filtered_df)

In [ ]:
# convert filtered actuators to a python dict

filtered_df.to_dict(orient='records')

#### 2.4 list all variables - (meters)

In [ ]:
# Electricity:Facility, Cooling:EnergyTransfer, Fan energy, etc...

util.list_available_meters()

## ----- 3. Run simulation -----

#### 3.1

In [ ]:

# util.ensure_output_variables([
#     {"name":"Zone Air System Sensible Cooling Energy", "key":"*", "freq":"TimeStep"},
#     {"name":"Zone Total Internal Latent Gain Energy", "key":"*", "freq":"TimeStep"},
#     {"name": "Zone Air CO2 Concentration", "key": "*", "freq": "TimeStep"},
#     {"name": "Zone Outdoor Air Inlet Mass Flow Rate", "key": "*", "freq": "TimeStep"},
#     {"name": "System Node Standard Density Volume Flow Rate", "key": "*", "freq": "TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE2-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE3-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE4-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE5-1", "freq":"TimeStep"},
# ], activate=True)
# 1) Make sure SQL will be produced
# util.ensure_output_sqlite(activate=True)

# site_additional_vars = [
#     "Site Wind Speed",
#     "Site Wind Direction",
#     "Site Diffuse Solar Radiation Rate per Area",
#     "Site Direct Solar Radiation Rate per Area",
#     "Site Horizontal Infrared Radiation Rate per Area",
#     "Site Sky Temperature",
# ]
# site_additional_specs=[{'name': v, 'key': 'Environment', 'freq': 'TimeStep'} for v in site_additional_vars]

#### 3.2 Select what variables/meters to save to eplusout.sql

In [ ]:
# `key="*"` → for all zones
# `freq="TimeStep"` → highest resolution

specs = [
    # --- Zone state + people ---
    {"name": "Zone Mean Air Temperature",                "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Dewpoint Temperature",       "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Air Relative Humidity",               "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Humidity Ratio",             "key": "*",            "freq": "TimeStep"},
    {"name": "Zone People Occupant Count",               "key": "*",            "freq": "TimeStep"},

    # --- CO₂ & OA into zones ---
    {"name": "Zone Air CO2 Concentration",               "key": "*",            "freq": "TimeStep"},

    # --- Site weather (Environment key) ---
    {"name": "Site Outdoor Air Drybulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Wetbulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Dewpoint Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Relative Humidity",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Humidity Ratio",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Barometric Pressure",        "key": "Environment", "freq": "TimeStep"},
    {"name": "Site Outdoor Air CO2 Concentration",                          "key": "Environment",  "freq": "TimeStep"},

    {"name": "System Node Temperature",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Mass Flow Rate",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Humidity Ratio",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node CO2 Concentration",               "key": "*",            "freq": "TimeStep"},
]

# 1) Ensure the Output:Variable objects exist (dedup-aware)
util.ensure_output_variables(specs, activate=True)


# 2) Ensure the meter(s) you want are reported
output_meters = ["InteriorLights:Electricity:Zone:SPACE5-1",
                 "Cooling:EnergyTransfer:Zone:SPACE1-1",
                 "Cooling:EnergyTransfer","Electricity:Facility",
                 "ElectricityPurchased:Facility",
                 "ElectricitySurplusSold:Facility"]
util.ensure_output_meters(output_meters, freq="TimeStep")


#### 3.3

In [ ]:
#3) Register callbacks
# util.register_handlers(
#     "after_hvac",
#     [{"method_name": "probe_zone_air_and_supply",
#       "key_wargs": {"log_every_minutes": 1, "precision": 3}}],
#     clear=False, run_during_warmup=False
# )

#### 3.4 Handler - occupancy

In [ ]:
# Runs after HVAC is solved each timestep
# Injects synthetic occupancy
# Used to test EKF’s ability to recover occupancy

util.register_handlers(
    "after_hvac",
    [{"method_name": "occupancy_handler",
      "key_wargs": {"lam": 33.0, "min": 20, "max": 45, "seed": 123}}],
    clear=False, run_during_warmup=False
)

#### 3.5 Handler - EKF

In [ ]:
# Runs EKF every timestep
# Estimates thermal parameters / moisture parameters / CO₂ parameters / occupancy
# save to eplusout.sql

util.register_handlers(
    "begin",
    [{"method_name": "probe_zone_air_and_supply_with_kf",
     "key_wargs": {
         "log_every_minutes": 15,
         "precision": 3,

         "kf_db_filename": "eplusout_kf_test.sqlite",
         "kf_batch_size": 50,
         "kf_commit_every_batches": 10,
         "kf_checkpoint_every_commits": 5,
         "kf_journal_mode": "WAL",
         "kf_synchronous": "NORMAL",

         # --- 10-state init (αo, αs, αe, βo, βs, βe, γe, Tz, wz, cz)
         "kf_init_mu":        [0.1, 0.1, 0.0,  0.1, 0.1, 0.0,  0.0,  20.0, 0.008, 400.0],
         "kf_init_cov_diag":  [1.0, 1.0, 1.0,  1.0, 1.0, 1.0,  1.0,  25.0, 1e-3,  1e3  ],
         "kf_sigma_P_diag":   [1e-6,1e-6,1e-6, 1e-6,1e-6,1e-6, 1e-6, 1e-5, 1e-6,  1e-4 ],

         # Optional: pretty column names for state persistence (dynamic schema)
         "kf_state_col_names": [
             "alpha_o","alpha_s","alpha_e","beta_o","beta_s","beta_e","gamma_e","Tz","wz","cz"
         ],

         # Use the 10-state EKF preparer
         "kf_prepare_fn": util._kf_prepare_inputs_zone_energy_model
     }}],
    clear=True
)

#### 3.6

In [ ]:
# util.register_handlers(
#     "before_hvac",
#     [{"method_name": "tick_set_actuator",
#       "kwargs": {
#         "component_type": "System Node Setpoint",
#         "control_type":   "Mass Flow Rate Setpoint",
#         "actuator_key":   "SPACE4-1 ZONE COIL AIR IN NODE",
#         "value":          0.35,            # kg/s request
#         "when":           "success",
#         "read_back":      True,            # read back actuator value after setting
#         "precision":      4
#       }}],
#     run_during_warmup=False
# )

#### 3.7

In [ ]:
# util.register_handlers(
#     "begin",   # or "after_hvac", etc.
#     [{"method_name": "tick_set_actuator",
#       "kwargs": {
#         "component_type": "People",
#         "control_type": "Number of People",
#         "actuator_key": "SPACE1-1 PEOPLE 1",
#         "value": 22.0,
#         "when": "success",
#         "read_back": True,
#         "precision": 3
#       }}],
#     run_during_warmup=False
# )

#### 3.8

In [ ]:
# util.register_handlers(
#     "after_hvac",
#     [{"method_name": "tick_log_actuator",
#       "kwargs": {
#         "component_type": "System Node Setpoint",
#         "control_type":   "Mass Flow Rate Setpoint",
#         "actuator_key":   "SPACE4-1 ZONE COIL AIR IN NODE",
#         "when": "always", #"on_change",
#         "precision": 3
#       }}],
#     run_during_warmup=False
# )


#### 3.9

In [ ]:
# util.register_handlers(
#     "begin",  # callback_begin_system_timestep_before_predictor
#     [{
#         "method_name": "tick_log_variable",
#         "kwargs": {
#             "name": "Zone People Occupant Count",
#             "key": "SPACE1-1",          # <-- replace with your zone name
#             "when": "always",        # log only when it changes
#             "precision": 0,             # people → integers are nice to see
#             "include_timestamp": True,
#             "allow_warmup": False
#         }
#     }],
#     clear=False,
#     run_during_warmup=False
# )

#### 3.10 Handler - energy meters

In [ ]:
# Logs facility electricity every timestep
# Produces clean time series for energy plots

util.register_handlers(
    "after_hvac",   # alias for callback_begin_system_timestep_before_predictor
    [{
        "method_name": "tick_log_meter",
        "kwargs": {
            "name": "Electricity:Facility",
            "which": "value",            # current tick value
            "when": "always",         # only log when it changes
            "precision": 3,
            "include_timestamp": True,
            "allow_warmup": False        # skip during sizing/warmup
        }
    }],
    clear=False,
    run_during_warmup=False # Skips warmup period
)

#### 3.11 Run full simulation

In [ ]:
# Runs full annual EnergyPlus simulation
# All callbacks execute in real time
# Outputs saved to:`eplusout.sql`

rc=util.run_annual()

## ----- 4 See what variables/actuators/meters avaialbe -----

#### 4.1 list all variables - to measure

In [ ]:
# Zone Mean Air Temperature / Zone Air CO₂ Concentration / System Node Mass Flow Rate / Outdoor air conditions

util.list_available_variables()

#### 4.2 list all actuators - to control

In [ ]:
# VAV damper flow setpoints / Schedule values / People object values / System node mass flow setpoints
# Includes Component type / Control type / Actuator key

util.list_available_actuators() #.to_dict(orient='records')

#### 4.3 list all meters - to read

In [ ]:
# Electricity:Facility / Cooling:EnergyTransfer / Heating:EnergyTransfer / Fan electricity

util.list_available_meters()

## ----- 5. See what's in the eplusout.sql file -----

#### 5.1 Create SQL explorer object

In [ ]:
# Opens the EnergyPlus results database
# Does NOT rerun the simulation
# Only reads stored results

xp = EPlusSqlExplorer(f"{out_dir}/eplusout.sql")

#### 5.2 List the tables in eplusout.sql

In [ ]:
# ReportData / ReportDataDictionary / Time / EnvironmentPeriods

xp.list_tables()

#### 5.3 Filter vairables in eplusout.sql

In [ ]:
# Filters for the variable "System Node Temperature"
# Returns variable name / key (which node) / units / reporting frequency / number of rows

# For which nodes do I actually have temperature data
df = xp.list_sql_variables(name="System Node Temperature")

# node name, how many timesteps data exists
df[['KeyValue','n_rows']].head(20)

#### 5.4 Save weather data from eplusout.sql to .csv file

In [ ]:
# Reads weather-related variables from `eplusout.sql`
# Saves them into a CSV file - eplus_out/weather_timeseries.csv

# Outdoor temperature / Humidity ratio / Relative humidity / CO₂ / Pressure / Solar data (if enabled)

util.export_weather_sql_to_csv()

#### 5.5 Convert weather csv to pandas df

In [ ]:
weather_df=pd.read_csv('eplus_out/weather_timeseries.csv')
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
weather_df['month'] = weather_df['timestamp'].dt.month

In [ ]:
weather_df

#### 5.6 See weather variable details and plot

In [ ]:
import numpy as np
from scipy.stats import norm, lognorm, gamma
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Extract the temperature data
variable='Site Outdoor Air Humidity Ratio [kgWater/kgDryAir]'#'Site Outdoor Air Dewpoint Temperature [C]' #'Site Outdoor Air Drybulb Temperature [C]' #
n=9
data_df = weather_df[weather_df['month']==n]
data = data_df[variable]

# Fit a normal distribution to the data:
mu, std = norm.fit(data)

# Create the histogram trace from the previous plot
counts, bin_edges = np.histogram(data, bins=50) # Adjust bin count as needed
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

histogram_trace = go.Bar(x=bin_centers, y=counts, name='Histogram', opacity=0.7)

# Create the Gaussian curve trace
xmin, xmax = data.min(), data.max()
x_norm = np.linspace(xmin, xmax, 100)
p_norm = norm.pdf(x_norm, mu, std)

# Scale the PDF to match the histogram's count scale
bin_width = bin_edges[1] - bin_edges[0]
scaled_pdf_norm = p_norm * len(data) * bin_width

gaussian_trace = go.Scatter(x=x_norm, y=scaled_pdf_norm, mode='lines', name=f'Gaussian Fit (μ={mu:.2f}, σ={std:.2f})', line=dict(color='red', width=2))

# Fit Log-Normal distribution
# Log-normal distribution requires positive data. Since temperature can be negative,
# a simple log-normal fit might not be appropriate directly.
# However, for demonstration, we can fit it to the positive part or shift the data.
# Let's fit it to the original data, understanding the limitations if negative values exist.
# We need to be careful if temperature_data contains zero or negative values for lognorm fit.
# For simplicity, we'll add an offset if there are non-positive values.
offset = 0
if (data <= 0).any():
    offset = -data.min() + 1 # Shift data to be positive
    print(f"Shifting data by {offset:.2f} for Log-Normal fit to ensure positivity.")

shape_lognorm, loc_lognorm, scale_lognorm = lognorm.fit(data + offset)

# Generate points for the fitted Log-Normal curve
# Ensure the x range is appropriate for the shifted data
x_lognorm = np.linspace(data.min() + offset, data.max() + offset, 100)
p_lognorm = lognorm.pdf(x_lognorm, shape_lognorm, loc_lognorm, scale_lognorm)

# Scale the PDF and shift x back for plotting
scaled_pdf_lognorm = p_lognorm * len(data) * bin_width
x_lognorm_unshifted = x_lognorm - offset

lognormal_trace = go.Scatter(x=x_lognorm_unshifted, y=scaled_pdf_lognorm, mode='lines', name=f'Log-Normal Fit', line=dict(color='green', width=2))


# Fit Gamma distribution
# Gamma distribution also typically requires positive data. Similar consideration as Log-Normal.
# We'll fit it to the shifted data if an offset was applied for lognormal.
shape_gamma, loc_gamma, scale_gamma = gamma.fit(data + offset)

# Generate points for the fitted Gamma curve
# Ensure the x range is appropriate for the shifted data
x_gamma = np.linspace(data.min() + offset, data.max() + offset, 100)
p_gamma = gamma.pdf(x_gamma, shape_gamma, loc_gamma, scale_gamma)

# Scale the PDF and shift x back for plotting
scaled_pdf_gamma = p_gamma * len(data) * bin_width
x_gamma_unshifted = x_gamma - offset


gamma_trace = go.Scatter(x=x_gamma_unshifted, y=scaled_pdf_gamma, mode='lines', name=f'Gamma Fit', line=dict(color='purple', width=2))


# Create the figure and add traces
fig = go.Figure()
fig.add_trace(histogram_trace)
fig.add_trace(gaussian_trace)
fig.add_trace(lognormal_trace)
fig.add_trace(gamma_trace)


# Update layout
fig.update_layout(title=f'Distribution of {variable} with Distribution Fits',
                  xaxis_title=variable,
                  yaxis_title='Count',
                  barmode='overlay' # Overlay bars to see fits better
                 )

# Show the plot
fig.show()

## ----- 6. Plot results from eplusout.sql file (zones) -----

#### 6.1 Zone mean air temp

In [ ]:
# Drybulb (auto-picks top zone keys if keys=None)
# Reads zone temperature data from SQL
# keys=["*"] → plot multiple zones automatically
# resample="1h" → average to hourly values
# temperature vs time plot

fig1=util.plot_sql_zone_variable(
    "Zone Mean Air Temperature",
    keys=["*"],                          # auto-pick a few zones with data
    reporting_freq=("TimeStep",),       # match how you logged
    resample="1h",
    title="Zone Mean Air Temperature"
)

#### 6.2 Zone mean air humidity ratio

In [ ]:
# Humidity ratio
fig2=util.plot_sql_zone_variable(
    "Zone Mean Air Humidity Ratio",
    keys=["*"],
    reporting_freq=("TimeStep",),
    resample="1h",
    title="Zone Mean Air Humidity Ratio"
)

#### 6.3 Zone Air CO₂ Concentration

In [ ]:
# Plots indoor air quality
# Shows CO₂ buildup and removal
# Reflects occupancy + ventilation

# CO2 concentration
fig3=util.plot_sql_zone_variable(
    "Zone Air CO2 Concentration",
    keys={"*"},
    reporting_freq=("TimeStep",),
    resample="1h",
    title="Zone Air CO2 Concentration"
)

## ----- 7. Plot results from eplusout.sql file (outdoor) -----

#### 7.1 Select what outdoor variables to plot

In [ ]:
# kind="var" → variable (not meter)
# key="Environment" → outdoor conditions

sels = [
    {"kind":"var", "name":"Site Outdoor Air Drybulb Temperature", "key":"Environment", "label":"Tdb [C]"},
    {"kind":"var", "name":"Site Outdoor Air Dewpoint Temperature", "key":"Environment", "label":"Tdew [C]"},
    {"kind":"var", "name":"Site Outdoor Air Humidity Ratio",      "key":"Environment", "label":"w [kg/kg]"},
]

#### 7.2 Plot outdoor weather data

In [ ]:
fig4=util.plot_sql_series(
    selections=sels,
    reporting_freq=("TimeStep",),
    include_design_days=False,
    resample="1h",
    meters_to_kwh=False,
    title="Outdoor (Environment)"
)

## ----- 8. Plot results from eplusout.sql file (Supply air) -----

#### 8.1 Per zone read nodes from sql file

In [ ]:
z2nodes = util._discover_zone_inlet_nodes_from_sql()

#### 8.2 Select a zone to inspect supply air

In [ ]:
zone = "SPACE4-1"

#### 8.3 Plot supply air mass flow rate of a  selected zone

In [ ]:
sels = [{"kind":"var", "name":"System Node Mass Flow Rate", "key":n, "label":n} for n in z2nodes[zone]]
fig5=util.plot_sql_series(
    selections=sels,
    reporting_freq=("TimeStep",),
    resample="15min",
    meters_to_kwh=False,
    title=f"{zone} — Supply Node Mass Flow Rate"
)

#### 8.4 Plot supply air temp of a  selected zone

In [ ]:
sels = [{"kind":"var", "name":"System Node Temperature", "key":n, "label":n} for n in z2nodes[zone]]
fig6=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node Temperature")

#### 8.5 Plot supply air humidity ratio of a selected zone

In [ ]:
sels = [{"kind":"var", "name":"System Node Humidity Ratio", "key":n, "label":n} for n in z2nodes[zone]]
fig7=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node Humidity Ratio")

#### 8.6 Plot supply air CO2 concentration of a selected zone

In [ ]:
sels = [{"kind":"var", "name":"System Node CO2 Concentration", "key":n, "label":n} for n in z2nodes[zone]]
fig8=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node CO2 Concentration")

## ----- 9. Plot results from eplusout.sql file (Occupancy)

#### 9.1 Find zones where occupancy data avialable

In [ ]:
# 1) discover zone keys that exist for the variable
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,            # don't filter; accept Zone Timestep, Hourly, etc.
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

# (optional) limit to first N zones
# occ_keys = occ_keys[:8]

#### 9.2 Plot per zone occupancy levels

In [ ]:
# Creates a list of plot instructions
# One time series per zone

selections = [
    {'kind':'var', 'name':'Zone People Occupant Count', 'key':k, 'label':k}
    for k in occ_keys
]

fig = util.plot_sql_series(
    selections=selections,
    reporting_freq=None,      # pull whatever is in the DB
    resample='1h',            # average to hourly; set to None for native timestep
    aggregate_vars='mean',    # hourly mean occupancy; use 'sum' for person-hours per hour
    title='Occupant Count per Zone',
    show=True
)

## ----- 10. Plot results from eplusout.sql file (Covariance) -----

#### 10.1 Find zones where occupancy data exists

In [ ]:
# 2) build selections using those keys (so keys and zones match the DB)
# output_sels = [
#     {'kind':'var','name':'Zone Mean Air Temperature','key':k,'label':f'MAT: {k}'}
#     for k in occ_keys
# ] + [
#     {'kind':'var','name':'Zone Air Relative Humidity','key':k,'label':f'ARH: {k}'}
#     for k in occ_keys
# ] +
# output_sels =  [
#     {'kind':'var','name':'Zone Air System Sensible Cooling Energy','key':k,'label':f'QSEN: {k}'}
#     for k in occ_keys
# ]
# + [
#     {'kind':'var','name':'Zone Total Internal Latent Gain Energy','key':k,'label':f'QLAT: {k}'}
#     for k in occ_keys
# ]


#### 10.2 Select what variable to compare (output / dep. variable (y))

In [ ]:
# let's compare CO2 concentration against occupancy

output_sels = [
    {'kind':'var','name':'Zone Air CO2 Concentration','key':k,'label':f'CO2: {k}'}
    for k in occ_keys
]

#### 10.2 Select what variable to compare (input / indep. variable (x))

In [ ]:
control_sels = (
    [{'kind':'var','name':'Zone People Occupant Count','key':k,'label':f'Occ: {k}'} for k in occ_keys]
    # + [
    #     {'kind':'var','name':'Site Outdoor Air Drybulb Temperature','key':'Environment','label':'OAT'},
    #     {'kind':'var','name':'Site Outdoor Air Wetbulb Temperature','key':'Environment','label':'OWB'},
    # ]
)

#### 10.3 Plot covariance heatmap dep. vs indep. variable

In [ ]:
# plot CO2 concetration (dep. variable) against occupancy (indep. variable)
fig = util.plot_sql_cov_heatmap(
    control_sels=control_sels,
    output_sels=output_sels,
    reporting_freq=None,     # <- don't filter out Zone Timestep rows
    resample='1h',           # compute cov on hourly series
    reduce='mean',
    stat='cov',              # or 'corr' if you want scale-free
    min_periods=12,
    include_design_days=False
)

## ----- 11. Plot results from eplusout_kf_test.sqlite. file (EKF) -----

#### 11.1 Find and open the EKF results database

In [ ]:
# Opens the SQLite file created by - probe_zone_air_and_supply_with_kf
# This DB is separate from `eplusout.sql`

xp=EPlusSqlExplorer(sql_path="eplus_out/eplusout_kf_test.sqlite")

#### 11.2 Load EKF estimates table

In [ ]:
# timestamps / zone names
# measurements
# predictions
# state estimates

df = xp.get_table_data(db="eplus_out/eplusout_kf_test.sqlite", table="KalmanEstimates")

#### 11.3 Filter EKF result for one zone

In [ ]:
zone1_df=df[df["Zone"]=="SPACE1-1"]

In [ ]:
df

In [ ]:

zone1_df

## extra